In [ ]:
import numpy as np
import pandas as pd
import scipy as sc
import csv
import plotly as py
import sklearn as skl
import nltk
import os
import io
import re
from nltk.corpus import stopwords
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.tree import DecisionTreeClassifier
from pandas import *
%matplotlib inline

In [ ]:
nltk.download()

In [ ]:
# texts 2013 holds all 365 days worth of news articles from the year 2013
texts2013 = []
location = 'D:\\Homework3\\2013'
for filename in os.listdir(location):
    fullfilename = location + '\\' + filename
    with open(fullfilename, 'r', encoding='latin-1') as myfile:
        text = myfile.read()
        texts2013.append(text)
        myfile.close()

In [ ]:
# texts 2014 holds all 365 days worth of news articles from the year 2014
texts2014 = []
location = 'D:\\Homework3\\2014'
for filename in os.listdir(location):
    fullfilename = location + '\\' + filename
    with open(fullfilename, 'r', encoding='latin-1') as myfile:
        text = myfile.read()
        texts2014.append(text)
        myfile.close()

In [ ]:
# reads in training datasets
ceosTraining = pd.read_csv('D:\\Homework3\\all\ceoEdited.csv', delimiter=',', encoding='latin-1', header=None,names=["CEO Training"])
companiesTraining = pd.read_csv('D:\\Homework3\\all\companiesEdited.csv', delimiter=',', encoding='latin-1', header=None,names=["Companies Training"])
percentTraining = pd.read_csv('D:\\Homework3\\all\percentage.csv', delimiter=',', encoding='latin-1', header=None,names=["Percentage Training"])
companySuffixes = pd.read_csv('D:\\Homework3\\all\companySuffixes.csv', delimiter=',', header=None,names=["Suffixes"])

In [ ]:
# we only want sentences where RegEx finds "Firstname Lastname", so we can isolate the sample
# firstLastSent is a collection of sentences with Firstname Lastname pattern, stop words removed

stop = set(stopwords.words('english'))
stop.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])
firstLastSent = []

for newsDate in np.arange(len(texts2013)):
    currentText = texts2013[newsDate]
    currentSentences = nltk.sent_tokenize(currentText)
    for j in np.arange(len(currentSentences)):
        thisSentence = currentSentences[j]
        m = re.search(rC,thisSentence)
        if (m != None):
            thisSentence = nltk.word_tokenize(thisSentence)
            thisSentence = [word for word in thisSentence if not word in stop]
            firstLastSent.append(thisSentence)
            
for newsDate in np.arange(len(texts2014)):
    currentText = texts2014[newsDate]
    currentSentences = nltk.sent_tokenize(currentText)
    for j in np.arange(len(currentSentences)):
        thisSentence = currentSentences[j]
        m = re.search(r"[A-Z][a-z]* [A-Z][a-z]*",thisSentence)
        if (m != None):
            thisSentence = nltk.word_tokenize(thisSentence)
            thisSentence = [word for word in thisSentence if not word in stop]
            firstLastSent.append(thisSentence)

In [ ]:
# creates trainingSample, used to train the classification model for CEOs
# we know approximately 18,000 sentences have CEO names, 230,000 have FirstName LastName
# select all sentences with CEO names, and 10% of sentences without
trainingSample = []

for i in np.arange(len(firstLastSent)):
    thisSentence = firstLastSent[i]
    selectSentence = 0
    for j in np.arange(len(thisSentence)-1):
        coupling = thisSentence[j] + " " + thisSentence[j+1]
        if (coupling in ceosTraining['CEO Training'].values):
            selectSentence = 1
    if ((selectSentence == 1) or (np.random.uniform() < .1)):
        trainingSample.append(thisSentence)

In [ ]:
# code finds how many sentences are in corpus
sentencesIn2013 = 0
for newsDate in np.arange(len(texts2013)):
    currentText = texts2013[newsDate]
    currentSentences = nltk.sent_tokenize(currentText)
    currentNumber = len(currentSentences)
    sentencesIn2013 = sentencesIn2013 + currentNumber

sentencesIn2014 = 0
for newsDate in np.arange(len(texts2014)):
    currentText = texts2014[newsDate]
    currentSentences = nltk.sent_tokenize(currentText)
    currentNumber = len(currentSentences)
    sentencesIn2014 = sentencesIn2014 + currentNumber
    
corpusSentences = sentencesIn2013 + sentencesIn2014

In [ ]:
Token = []
Coupling = []
AlphaNum = []
PrevCap = []
Title = []
NextCap = []
Position = []
FirstLength = []
SecondLength = []
FirstNoun = []
SecondNoun = []
CEOinSent = []
IsCEO = []

for sent in np.arange(len(trainingSample)):
    sentence = trainingSample[sent]
    sentencePOS = nltk.pos_tag(sentence)
    sentLength = len(sentence)
    
    for i in np.arange(sentLength):
        if (sentence[i].lower() == "ceo"):
            thisCEOinSent = True
        elif (i < sentLength - 2) and (sentence[i].lower() == "chief") and (sentence[i+1].lower() == "executive") and (sentence[i+2].lower() == "officer"):
            thisCEOinSent = True
        else:
            thisCEOinSent = False
            
    for word in np.arange(sentLength):
        if sentence[word].istitle():
            thisToken = sentence[word]
            Token.append(thisToken)

            if (word == sentLength - 1):
                thisCoupling = thisToken
                Coupling.append(thisCoupling)
                AlphaNum.append(thisToken.isalpha())
                SecondLength.append(False)
                SecondPOS = False
            else:
                thisCoupling = thisToken + " " + sentence[word+1]
                Coupling.append(thisCoupling)
                AlphaNum.append(thisToken.isalpha() and sentence[word+1].isalpha())
                SecondLength.append(len(sentence[word+1]))
                SecondPOS = sentencePOS[word+1][1]

            if (word==0):
                PrevCap.append(False)
                #PrevPOS = False
            else:
                PrevCap.append(sentence[word-1].istitle())
                #PrevPOS = sentencePOS[word-1][1]

            if thisCoupling.istitle():
                Title.append(True)
            else:
                Title.append(False)

            if (word >= (sentLength-2) or (not sentence[word+2].istitle())):
                NextCap.append(False)
            else:
                NextCap.append(True)

            #if (word >= sentLength - 2):
            #    NextPOS = False
            #else:
            #    NextPOS = sentencePOS[word+2][1]

            Position.append(word)
            FirstLength.append(len(thisToken))
            FirstPOS = sentencePOS[word][1]

            if (FirstPOS == "NNP") or (FirstPOS == "NN"):
                FirstNoun.append(True)
            else:
                FirstNoun.append(False)

            if (SecondPOS == "NNP") or (SecondPOS == "NN"):
                SecondNoun.append(True)
            else:
                SecondNoun.append(False)
                
            CEOinSent.append(thisCEOinSent)

            if (thisCoupling in ceosTraining['CEO Training'].values):
                IsCEO.append(True)
            else:
                IsCEO.append(False)

In [ ]:
train = np.asarray([PrevCap, Title, Position, FirstLength, SecondLength, NextCap, FirstNoun, SecondNoun, CEOinSent])
trainLabels = IsCEO
train = train.transpose()

In [ ]:
clf = tree.DecisionTreeClassifier()
ynb = clf.fit(train, trainLabels)

In [ ]:
predicted = ynb.predict(train)

In [ ]:
np.sum((predicted == trainLabels))
np.sum(predicted)

In [ ]:
#index = [i for i, x in enumerate(trainLabels) if x == True]
#CEOdata['FirstNoun'].values[index]

In [ ]:
len(firstLastSent)

In [ ]:
# collect data on entire firstLastSent sample

Token = []
Coupling = []
AlphaNum = []
PrevCap = []
Title = []
NextCap = []
Position = []
FirstLength = []
SecondLength = []
FirstNoun = []
SecondNoun = []
CEOinSent = []
IsCEO = []

for sent in np.arange(len(firstLastSent)):
    sentence = firstLastSent[sent]
    sentencePOS = nltk.pos_tag(sentence)
    sentLength = len(sentence)
    
    for i in np.arange(sentLength):
        if (sentence[i].lower() == "ceo"):
            thisCEOinSent = True
        elif (i < sentLength - 2) and (sentence[i].lower() == "chief") and (sentence[i+1].lower() == "executive") and (sentence[i+2].lower() == "officer"):
            thisCEOinSent = True
        else:
            thisCEOinSent = False
            
    for word in np.arange(sentLength):
        if sentence[word].istitle():
            thisToken = sentence[word]
            Token.append(thisToken)

            if (word == sentLength - 1):
                thisCoupling = thisToken
                Coupling.append(thisCoupling)
                AlphaNum.append(thisToken.isalpha())
                SecondLength.append(False)
                SecondPOS = False
            else:
                thisCoupling = thisToken + " " + sentence[word+1]
                Coupling.append(thisCoupling)
                AlphaNum.append(thisToken.isalpha() and sentence[word+1].isalpha())
                SecondLength.append(len(sentence[word+1]))
                SecondPOS = sentencePOS[word+1][1]

            if (word==0):
                PrevCap.append(False)
                #PrevPOS = False
            else:
                PrevCap.append(sentence[word-1].istitle())
                #PrevPOS = sentencePOS[word-1][1]

            if thisCoupling.istitle():
                Title.append(True)
            else:
                Title.append(False)

            if (word >= (sentLength-2) or (not sentence[word+2].istitle())):
                NextCap.append(False)
            else:
                NextCap.append(True)

            #if (word >= sentLength - 2):
            #    NextPOS = False
            #else:
            #    NextPOS = sentencePOS[word+2][1]

            Position.append(word)
            FirstLength.append(len(thisToken))
            FirstPOS = sentencePOS[word][1]

            if (FirstPOS == "NNP") or (FirstPOS == "NN"):
                FirstNoun.append(True)
            else:
                FirstNoun.append(False)

            if (SecondPOS == "NNP") or (SecondPOS == "NN"):
                SecondNoun.append(True)
            else:
                SecondNoun.append(False)
                
            CEOinSent.append(thisCEOinSent)

            if (thisCoupling in ceosTraining['CEO Training'].values):
                IsCEO.append(True)
            else:
                IsCEO.append(False)

In [ ]:
test = np.asarray([PrevCap, Title, Position, FirstLength, SecondLength, NextCap, FirstNoun, SecondNoun, CEOinSent])
test = test.transpose()
predicted = ynb.predict(test)

In [ ]:
FP = 0
FN = 0
TP = 0
TN = 0
FNarray = np.zeros(len(IsCEO))

for i in np.arange(len(predicted)):
    thisPredict = predicted[i]
    thisActual = IsCEO[i]
    if thisPredict and thisActual:
        TP = TP + 1
    elif thisPredict and (thisActual == False):
        FP = FP + 1
    elif (thisPredict == False) and thisActual:
        FN = FN + 1
    elif (thisPredict == False) and (thisActual == False):
        TN = TN + 1

In [ ]:
print(str(FP) + " false positives\n" + str(FN) + " false negatives\n" + str(TP) + " true positives\n" + str(TN) + " true negatives")
print("\naccuracy is " + str((TP + TN)/(TP + TN + FP + FN)))
print("precision is " + str(TP/(TP + FP)))
print("recall is " + str(TP/(TP + FN)))

In [ ]:
predictedCEOs = []

for i in np.arange(len(predicted)):
    if (predicted[i] == True):
        predictedCEOs.append(Coupling[i])

# following function used from Markus Jarderot on StackOverflow
# http://stackoverflow.com/questions/480214/how-do-you-remove-duplicates-from-a-list-in-whilst-preserving-order
def f7(seq):
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

CEOsList = f7(predictedCEOs)

In [ ]:
outputFile = open("./ShipleyPredictedCEOs.txt", "w", encoding = 'latin-1')
for item in CEOsList:
    outputFile.write("%s\n" % item)

In [ ]:
# EXTRACT PERCENTAGES
# We know that a percentage will have either the percent sign (%) or the word percent, extract these sentences
stop = set(stopwords.words('english'))
stop.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])

percentSent = []

for newsDate in np.arange(len(texts2013)):
    currentText = texts2013[newsDate]
    currentSentences = nltk.sent_tokenize(currentText)
    for j in np.arange(len(currentSentences)):
        thisSentence = currentSentences[j]
        m = re.search(r"\%",thisSentence)
        n = re.search(r"percent",thisSentence)
        if (m != None) or (n != None):
            thisSentence = nltk.word_tokenize(thisSentence)
            thisSentence = [word for word in thisSentence if not word in stop]
            percentSent.append(thisSentence)
            
for newsDate in np.arange(len(texts2014)):
    currentText = texts2014[newsDate]
    currentSentences = nltk.sent_tokenize(currentText)
    for j in np.arange(len(currentSentences)):
        thisSentence = currentSentences[j]
        m = re.search(r"\%",thisSentence)
        n = re.search(r"percent",thisSentence)
        if (m != None) or (n != None):
            thisSentence = nltk.word_tokenize(thisSentence)
            thisSentence = [word for word in thisSentence if not word in stop]
            percentSent.append(thisSentence)

In [ ]:
# creates trainingSample, used to train the classification model for percentages
trainingSample = []

for i in np.arange(len(percentSent)):
    if (np.random.uniform() < .2):
        trainingSample.append(percentSent[i])

In [ ]:
pToken = []
pCoupling = []
pFirstNoun = []
pFirstAdj = []
pFirstCD = []
pSecondNoun = []
pSecondPerc = []
pSecondPercSymb = []
pIsPercent = []

for sent in np.arange(len(trainingSample)):
    sentence = trainingSample[sent]
    sentencePOS = nltk.pos_tag(sentence)
    sentLength = len(sentence)
            
    for word in np.arange(sentLength-1):
        thisToken = sentence[word]
        nextToken = sentence[word+1]
        pToken.append(thisToken)
        
        if (nextToken == "%"):
            thisCoupling = thisToken + nextToken
        else:
            thisCoupling = thisToken + " " + nextToken
        pCoupling.append(thisCoupling)
        
        FirstPOS = sentencePOS[word][1]
        SecondPOS = sentencePOS[word+1][1]

        pFirstNoun.append(FirstPOS == "NN")
        pFirstAdj.append(FirstPOS == "JJ")
        pFirstCD.append(FirstPOS == "CD")
        pSecondNoun.append(SecondPOS == "NN")
        pSecondPerc.append(nextToken.lower() == "percent")
        pSecondPercSymb.append(nextToken == "%")
        
        if (thisCoupling in percentTraining['Percentage Training'].values):
            pIsPercent.append(True)
        else:
            pIsPercent.append(False)

In [ ]:
ptrain = np.asarray([pFirstNoun, pFirstAdj, pFirstCD, pSecondNoun, pSecondPerc, pSecondPercSymb])
ptrain = ptrain.transpose()

In [ ]:
clf = tree.DecisionTreeClassifier()
ynb = clf.fit(ptrain, pIsPercent)

In [ ]:
ppredicted = ynb.predict(ptrain)

In [ ]:
ptToken = []
ptCoupling = []
ptFirstNoun = []
ptFirstAdj = []
ptFirstCD = []
ptSecondNoun = []
ptSecondPerc = []
ptSecondPercSymb = []
ptIsPercent = []

for sent in np.arange(len(percentSent)):
    sentence = percentSent[sent]
    sentencePOS = nltk.pos_tag(sentence)
    sentLength = len(sentence)
            
    for word in np.arange(sentLength-1):
        thisToken = sentence[word]
        nextToken = sentence[word+1]
        ptToken.append(thisToken)
        
        if (nextToken == "%"):
            thisCoupling = thisToken + nextToken
        else:
            thisCoupling = thisToken + " " + nextToken
        ptCoupling.append(thisCoupling)
        
        FirstPOS = sentencePOS[word][1]
        SecondPOS = sentencePOS[word+1][1]

        ptFirstNoun.append(FirstPOS == "NN")
        ptFirstAdj.append(FirstPOS == "JJ")
        ptFirstCD.append(FirstPOS == "CD")
        ptSecondNoun.append(SecondPOS == "NN")
        ptSecondPerc.append(nextToken.lower() == "percent")
        ptSecondPercSymb.append(nextToken == "%")
        
        if (thisCoupling in percentTraining['Percentage Training'].values):
            ptIsPercent.append(True)
        else:
            ptIsPercent.append(False)

In [ ]:
pttest = np.asarray([ptFirstNoun, ptFirstAdj, ptFirstCD, ptSecondNoun, ptSecondPerc, ptSecondPercSymb])
pttest = pttest.transpose()
ptpredicted = ynb.predict(pttest)

In [ ]:
sum(ptIsPercent)

In [ ]:
FP = 0
FN = 0
TP = 0
TN = 0

for i in np.arange(len(ptpredicted)):
    thisPredict = ptpredicted[i]
    thisActual = ptIsPercent[i]
    if thisPredict and thisActual:
        TP = TP + 1
    elif thisPredict and (thisActual == False):
        FP = FP + 1
    elif (thisPredict == False) and thisActual:
        FN = FN + 1
    elif (thisPredict == False) and (thisActual == False):
        TN = TN + 1

In [ ]:
print(str(FP) + " false positives\n" + str(FN) + " false negatives\n" + str(TP) + " true positives\n" + str(TN) + " true negatives")
print("\naccuracy is " + str((TP + TN)/(TP + TN + FP + FN)))
print("precision is " + str(TP/(TP + FP)))
print("recall is " + str(TP/(TP + FN)))

In [ ]:
predictedPercents = []

for i in np.arange(len(ptpredicted)):
    if (ptpredicted[i] == True):
        predictedPercents.append(ptCoupling[i])

PercentsList = f7(predictedPercents)

In [ ]:
outputFile = open("./ShipleyPredictedPercents.txt", "w", encoding = 'latin-1')
for item in PercentsList:
    outputFile.write("%s\n" % item)

In [ ]:
# EXTRACT SENTENCES WITH COMMON COMPANY SUFFIXES
# We know that a percentage will have either the percent sign (%) or the word percent, extract these sentences

stop = set(stopwords.words('english'))
stop.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}'])

suffixes = re.compile('| '.join(companySuffixes['Suffixes']))
companySent = []

for newsDate in np.arange(len(texts2013)):
    currentText = texts2013[newsDate]
    currentSentences = nltk.sent_tokenize(currentText)
    for j in np.arange(len(currentSentences)):
        thisSentence = currentSentences[j]
        m = re.search(suffixes, thisSentence)
        if (m != None):
            thisSentence = nltk.word_tokenize(thisSentence)
            thisSentence = [word for word in thisSentence if not word in stop]
            companySent.append(thisSentence)
            
for newsDate in np.arange(len(texts2014)):
    currentText = texts2014[newsDate]
    currentSentences = nltk.sent_tokenize(currentText)
    for j in np.arange(len(currentSentences)):
        thisSentence = currentSentences[j]
        m = re.search(suffixes, thisSentence)
        if (m != None):
            thisSentence = nltk.word_tokenize(thisSentence)
            thisSentence = [word for word in thisSentence if not word in stop]
            companySent.append(thisSentence)

In [ ]:
# creates trainingSample, used to train the classification model for percentages
trainingSample = []

for i in np.arange(len(companySent)):
    if (np.random.uniform() < .2):
        trainingSample.append(companySent[i])

In [ ]:
Token = []
Grouping = []
Title = []
EndSuffix = []
PrevCap = []
IsAlpha = []
Subset = []
IsCompany = []

suf = companySuffixes["Suffixes"].tolist()
compTrainList = companiesTraining['Companies Training'].tolist()

for sent in np.arange(len(trainingSample)):
    sentence = trainingSample[sent]
    sentLength = len(sentence)
    
    for word in np.arange(sentLength-2):
        
        Token1 = sentence[word]
        
        if (word <= sentLength - 4):
            Token2 = sentence[word+1]
            Token3 = sentence[word+2]
            Token4 = sentence[word+3]
            
            Grouping4 = Token1 + " " + Token2 + " " + Token3 + " " + Token4
            Title4 = Grouping4.istitle()
            EndSuffix4 = (Token4 in suf)
            if (word == 0):
                PrevCap4 = False
            else:
                PrevCap4 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping4)
            Title.append(Title4)
            EndSuffix.append(EndSuffix4)
            PrevCap.append(PrevCap4)
            IsCompany.append(Grouping4 in compTrainList)
            Subset.append(False)
            
            Grouping3 = Token1 + " " + Token2 + " " + Token3
            Title3 = Grouping3.istitle()
            EndSuffix3 = (Token3 in suf)
            if (word == 0):
                PrevCap3 = False
            else:
                PrevCap3 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha() and Token3.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping3)
            Title.append(Title3)
            EndSuffix.append(EndSuffix3)
            PrevCap.append(PrevCap3)
            IsCompany.append(Grouping3 in compTrainList)
            Subset.append(Title4)
            
            Grouping2 = Token1 + " " + Token2
            Title2 = Grouping2.istitle()
            EndSuffix2 = (Token2 in suf)
            if (word == 0):
                PrevCap2 = False
            else:
                PrevCap2 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha() and Token3.isalpha() and Token4.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping2)
            Title.append(Title2)
            EndSuffix.append(EndSuffix2)
            PrevCap.append(PrevCap2)
            IsCompany.append(Grouping2 in compTrainList)
            Subset.append(Title4 or Title3)
            
        elif (word <= sentLength - 3):
            Token2 = sentence[word+1]
            Token3 = sentence[word+2]
            
            Grouping3 = Token1 + " " + Token2 + " " + Token3
            Title3 = Grouping3.istitle()
            EndSuffix3 = (Token3 in suf)
            if (word == 0):
                PrevCap3 = False
            else:
                PrevCap3 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping3)
            Title.append(Title3)
            EndSuffix.append(EndSuffix3)
            PrevCap.append(PrevCap3)
            IsCompany.append(Grouping3 in compTrainList)
            Subset.append(False)
            
            Grouping2 = Token1 + " " + Token2
            Title2 = Grouping2.istitle()
            EndSuffix2 = (Token2 in suf)
            if (word == 0):
                PrevCap2 = False
            else:
                PrevCap2 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha() and Token3.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping2)
            Title.append(Title2)
            EndSuffix.append(EndSuffix2)
            PrevCap.append(PrevCap2)
            IsCompany.append(Grouping2 in compTrainList)
            Subset.append(Title3)
            
        else:
            Token2 = sentence[word+1]
            Grouping2 = Token1 + " " + Token2
            Title2 = Grouping2.istitle()
            EndSuffix2 = (Token2 in suf)
            if (word == 0):
                PrevCap2 = False
            else:
                PrevCap2 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping2)
            Title.append(Title2)
            EndSuffix.append(EndSuffix2)
            PrevCap.append(PrevCap2)
            IsCompany.append(Grouping2 in compTrainList)
            Subset.append(False)

In [ ]:
TitleSubset = np.zeros(len(Grouping))
for i in np.arange(5, len(Grouping)-6):
    for j in [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5]:
        if (Grouping[i] in Grouping[i+j]) and Title[i+j]:
            TitleSubset[i] = True

In [ ]:
ctrain = np.asarray([Title, EndSuffix, PrevCap, IsAlpha, TitleSubset])
ctrain = ctrain.transpose()
clf = BernoulliNB()
ynb = clf.fit(ctrain, IsCompany)
cpredicted = ynb.predict(ctrain)

In [ ]:
Token = []
Grouping = []
Title = []
EndSuffix = []
PrevCap = []
IsAlpha = []
Subset = []
IsCompany = []

suf = companySuffixes["Suffixes"].tolist()
compTrainList = companiesTraining['Companies Training'].tolist()

for sent in np.arange(len(companySent)):
    sentence = companySent[sent]
    sentLength = len(sentence)
    
    for word in np.arange(sentLength-2):
        
        Token1 = sentence[word]
        
        if (word <= sentLength - 4):
            Token2 = sentence[word+1]
            Token3 = sentence[word+2]
            Token4 = sentence[word+3]
            
            Grouping4 = Token1 + " " + Token2 + " " + Token3 + " " + Token4
            Title4 = Grouping4.istitle()
            EndSuffix4 = (Token4 in suf)
            if (word == 0):
                PrevCap4 = False
            else:
                PrevCap4 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping4)
            Title.append(Title4)
            EndSuffix.append(EndSuffix4)
            PrevCap.append(PrevCap4)
            IsCompany.append(Grouping4 in compTrainList)
            Subset.append(False)
            
            Grouping3 = Token1 + " " + Token2 + " " + Token3
            Title3 = Grouping3.istitle()
            EndSuffix3 = (Token3 in suf)
            if (word == 0):
                PrevCap3 = False
            else:
                PrevCap3 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha() and Token3.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping3)
            Title.append(Title3)
            EndSuffix.append(EndSuffix3)
            PrevCap.append(PrevCap3)
            IsCompany.append(Grouping3 in compTrainList)
            Subset.append(Title4)
            
            Grouping2 = Token1 + " " + Token2
            Title2 = Grouping2.istitle()
            EndSuffix2 = (Token2 in suf)
            if (word == 0):
                PrevCap2 = False
            else:
                PrevCap2 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha() and Token3.isalpha() and Token4.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping2)
            Title.append(Title2)
            EndSuffix.append(EndSuffix2)
            PrevCap.append(PrevCap2)
            IsCompany.append(Grouping2 in compTrainList)
            Subset.append(Title4 or Title3)
            
        elif (word <= sentLength - 3):
            Token2 = sentence[word+1]
            Token3 = sentence[word+2]
            
            Grouping3 = Token1 + " " + Token2 + " " + Token3
            Title3 = Grouping3.istitle()
            EndSuffix3 = (Token3 in suf)
            if (word == 0):
                PrevCap3 = False
            else:
                PrevCap3 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping3)
            Title.append(Title3)
            EndSuffix.append(EndSuffix3)
            PrevCap.append(PrevCap3)
            IsCompany.append(Grouping3 in compTrainList)
            Subset.append(False)
            
            Grouping2 = Token1 + " " + Token2
            Title2 = Grouping2.istitle()
            EndSuffix2 = (Token2 in suf)
            if (word == 0):
                PrevCap2 = False
            else:
                PrevCap2 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha() and Token3.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping2)
            Title.append(Title2)
            EndSuffix.append(EndSuffix2)
            PrevCap.append(PrevCap2)
            IsCompany.append(Grouping2 in compTrainList)
            Subset.append(Title3)
            
        else:
            Token2 = sentence[word+1]
            Grouping2 = Token1 + " " + Token2
            Title2 = Grouping2.istitle()
            EndSuffix2 = (Token2 in suf)
            if (word == 0):
                PrevCap2 = False
            else:
                PrevCap2 = sentence[word-1].isupper()
            IsAlpha.append(Token1.isalpha() and Token2.isalpha())
            Token.append(Token1)
            Grouping.append(Grouping2)
            Title.append(Title2)
            EndSuffix.append(EndSuffix2)
            PrevCap.append(PrevCap2)
            IsCompany.append(Grouping2 in compTrainList)
            Subset.append(False)

In [ ]:
TitleSubset = np.zeros(len(Grouping))
for i in np.arange(5, len(Grouping)-6):
    for j in [-5, -4, -3, -2, -1, 1, 2, 3, 4, 5]:
        if (Grouping[i] in Grouping[i+j]) and Title[i+j]:
            TitleSubset[i] = True

In [ ]:
ctest = np.asarray([Title, EndSuffix, PrevCap, IsAlpha, TitleSubset])
ctest = ctest.transpose()
ctpredicted = ynb.predict(ctest)

In [ ]:
FP = 0
FN = 0
TP = 0
TN = 0

for i in np.arange(len(ctpredicted)):
    thisPredict = ctpredicted[i]
    thisActual = IsCompany[i]
    if thisPredict and thisActual:
        TP = TP + 1
    elif thisPredict and (thisActual == False):
        FP = FP + 1
    elif (thisPredict == False) and thisActual:
        FN = FN + 1
    elif (thisPredict == False) and (thisActual == False):
        TN = TN + 1

In [ ]:
print(str(FP) + " false positives\n" + str(FN) + " false negatives\n" + str(TP) + " true positives\n" + str(TN) + " true negatives")
print("\naccuracy is " + str((TP + TN)/(TP + TN + FP + FN)))
print("precision is " + str(TP/(TP + FP)))
print("recall is " + str(TP/(TP + FN)))

In [ ]:
predictedCompanies = []

for i in np.arange(len(ctpredicted)):
    if (ctpredicted[i] == True):
        predictedCompanies.append(Grouping[i])

CompanyList = f7(predictedCompanies)

In [ ]:
outputFile = open("./ShipleyPredictedCompanies.txt", "w", encoding = 'latin-1')
for item in CompanyList:
    outputFile.write("%s\n" % item)